## Objetivo del Notebook

Se quiere mejorar el modelo antiguo de predicción para el cuadl míñimo, este modelo se basaba en predecir el comportamiento de de los mínimos mensuales, sin emabrgo lo que interesa es predecir el valor mínimo de esta serie de tiempo por lo que se cambiará la metodología a una que involucre Teoría de Valores Extremos (EVT).

### Librerías Necesarias

En este caso se utilizará PyTorch para la creación del modelo Gated Recurrent Units (GRU) aprovechando del GPU para acelerar el proceso mediante CUDA.

In [7]:
## Librerias

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, f1_score
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [10]:
## Activar GPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(0))

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


### Metodología

Se usará la metodología de Teoría de Valores Extremos propuesta en el paper Modeling Extreme Values in Time Series sin embargo se usará únicamente el umbral del percentil 5% para predecir únicamente los mínimos. 
Se harán clases internas y funciones para facilitar la creación del modelo y su posterior entrenamiento y evaluación.

### Clase = Serie_Tiempo

El objetivo de esta clase es la manipulación de la serie de tiempo, tal y como se menciona en el paper, donde se establecen los parámetros como el umbral, la base de datos, el tamaño de la ventana y se crea la sucesión de sucesiones para los datos.

In [11]:
class Serie_Tiempo(Dataset):

    def __init__(self, data, tamano_ventana, pasos_prediccion, umbral, train = True):

        self.data = data
        self.tamano_ventana = tamano_ventana
        self.pasos_prediccion = pasos
        self.umbral = umbral
        self.train = train
        self.scaler = StandardScaler()

        if self.train:
            
            self.scaler.fit(self.data)
        
        self.data_normalizada = self.scaler.transform(data)
        self.X, self.Y, self.V = self.crear_sucesiones()

    def crear_sucesiones(self):

        X, Y, V = [], [], []

        for i in range(len(self.data_normalizada) - self.tamano_ventana - self.pasos_prediccion):

            X_sucesion = self.data_normalizad[i:i+self.tamano_ventana]
            Y_predecir = self.data_normalizada[i+self.tamano_ventana + self.pasos_prediccion -1]
            V_extremo = 1 if Y_predecir < -self.umbral else 0

            X.append(X_sucesion)
            Y.append(Y_predecir)
            V.append(V_extremo)

        return np.array(X), np.array(Y), np.array(V)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return (torch.FloatTensor(self.X[idx]).to(device),
                torch.FloatTensor([self.y[idx]]).to(device),
                torch.FloatTensor([self.v[idx]]).to(device))

### Clase = GatedRecurrentUnits

Como se menciona en el paper, se usará un modelo GRU para la predicción de los valores y mediante un índice de pérdida (definido en la anterior clase) se mejorará el modelo.

In [12]:
class GatedRecurrentUnits(nn.Module):

    def __init__(self, tamano_entrada, tamano_oculto, tamano_memoria, tamano_ventanas, gamma = 2.0):

        super(GatedRecurrentUnits, self).__init__()

        self.tamano_oculto = tamano_oculto
        self.tamano_memoria = tamano_memoria
        self.gamma = gamma
        self.tamano_ventanas = tamano_ventanas

        self.gru = nn.GRU(input_size = tamano_entrada, hidden_size = tamano_oculto, batch_first = True, num_layers = 2, dropout  = 0.2)

        self.memoria_gru = nn.GRU(input_size = tamano_entrada, hidden_size = tamano_oculto, batch_first = True)

        self.memoria_S = nn.Parameter(torch.randn(tamano_memoria, tamano_oculto)*0.1)
        self.memoria_Q = nn.Parameter(torch.zeros(tamano_memoria))

        self.pesos = nn.Linear(tamano_oculto, tamano_oculto)
        self.capa_salida = nn.Linear(tamano_oculto, 1)
        self.parametro_escala = nn.Parameter(torch.tensor(1.0))

        self.evento_predictor = nn.Sequential(
            nn.Linear(tamano_oculto, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,1),
            nn.Sigmoid()
        )

        self._inicializar_pesos()

    def _inicializar_pesos(self):

        for nombre, param in self.named_parameters():

            if 'weight' in nombre:
                nn.init.xavier_unifoirm_(param)
            elif 'bias' in nombre:
                nn.init.constant_(param,0)

    def forward(self, x, memoria_actualizar = False, ventanas_memoria = None):

        batch_size = x.size(0)

        gru_out, _ = self.gru(x)
        estado_oculto = gru_out[:, -1, :]

        base_pred = self.capa_salida(estado_oculto)

        scores_atencion = torch.matmul(estado_oculto, self.memoria_S.t())
        pesos_atencion = torch.softmax(scores_atencion, dim=-1)

        prediccion_evento = torch.matmul(pesos_atencion, torch.sigmoid(self.memoria_Q))
        prediccion_evento = torch.sigmoid(prediccion_evento)

        prediccion_final = base_pred - self.parametro_escala * prediccion_evento

        ventanas_memoria = None
        if ventanas_memoria is not None:

            ventanas_features = []

            for ventana in ventanas_memoria:

                _, ventana_oculta = self.memoria_gru(ventana.unsqueeze(0))
                ventanas_features.append(ventana_oculta.squeeze(0))

            ventanas_features = torch.stack(ventanas_features)
            ventanas_prediccion = self.evento_predictor(ventanas_features)
        
        return prediccion_final, base_pred, evento_predictor, ventanas_prediccion

In [ ]:
def extreme_value_loss(u, v, gamma=2.0, beta0=0.9, beta1=0.1):
    """
    Pérdida para eventos extremos de mínimo
    u: predicciones de probabilidad de evento
    v: etiquetas reales (0 o 1)
    """
    # Término de peso basado en Extreme Value Theory
    weight_term = torch.clamp(1 - u / gamma, min=1e-8) ** gamma
    
    # Pérdida para eventos positivos (mínimos extremos)
    loss_positive = -beta0 * weight_term * v * torch.log(u + 1e-8)
    
    # Pérdida para eventos negativos (normales)
    loss_negative = -beta1 * (1 - v) * torch.log(1 - u + 1e-8)
    
    return (loss_positive + loss_negative).mean()

def train_model(model, train_loader, val_loader, num_epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
    mse_loss = nn.MSELoss()
    
    train_losses, val_losses = [], []
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for batch_idx, (X_batch, y_batch, v_batch) in enumerate(train_loader):
            optimizer.zero_grad()
            
            # Forward pass
            final_pred, base_pred, event_pred, _ = model(X_batch)
            
            # Calcular pérdidas
            loss_mse = mse_loss(final_pred.squeeze(), y_batch.squeeze())
            loss_evl = extreme_value_loss(event_pred.squeeze(), v_batch.squeeze())
            
            # Pérdida total
            loss = loss_mse + 0.5 * loss_evl
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds, val_targets, val_events = [], [], []
        
        with torch.no_grad():
            for X_batch, y_batch, v_batch in val_loader:
                final_pred, base_pred, event_pred, _ = model(X_batch)
                
                loss_mse = mse_loss(final_pred.squeeze(), y_batch.squeeze())
                loss_evl = extreme_value_loss(event_pred.squeeze(), v_batch.squeeze())
                loss = loss_mse + 0.5 * loss_evl
                
                val_loss += loss.item()
                val_preds.extend(final_pred.cpu().numpy())
                val_targets.extend(y_batch.cpu().numpy())
                val_events.extend((event_pred > 0.5).cpu().numpy().astype(int))
        
        # Calcular métricas
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        val_rmse = np.sqrt(mean_squared_error(val_targets, val_preds))
        
        # Calcular F1-score para eventos extremos
        val_true_events = [1 if target < -0.5 else 0 for target in val_targets]  # Asumiendo epsilon=0.5
        val_f1 = f1_score(val_true_events, val_events, zero_division=0)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch:3d}/{num_epochs}: Train Loss: {train_loss:.4f}, '
                  f'Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}, Val F1: {val_f1:.4f}')
        
        # Guardar mejor modelo
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
    
    return train_losses, val_losses

# Parámetros de configuración
config = {
    'window_size': 50,
    'prediction_steps': 1,
    'hidden_size': 128,
    'memory_size': 80,
    'batch_size': 32,
    'num_epochs': 100,
    'learning_rate': 0.0005,
    'epsilon': 0.5,  # Umbral para mínimos extremos (ajustar según tus datos)
    'gamma': 2.0,    # Parámetro EVT
}

# Ejemplo de uso
def main():
    # 1. Cargar tus datos (reemplaza con tus datos reales)
    # data = pd.read_csv('tu_archivo.csv')['caudal'].values.reshape(-1, 1)
    
    # Datos de ejemplo (simulados)
    np.random.seed(42)
    n_samples = 1000
    time = np.linspace(0, 20, n_samples)
    data = np.sin(time) + 0.1 * np.random.randn(n_samples)
    
    # Añadir algunos mínimos extremos
    extreme_indices = [200, 450, 700, 850]
    data[extreme_indices] -= 2.5  # Mínimos extremos
    
    data = data.reshape(-1, 1)
    
    # 2. Dividir datos
    train_size = int(0.7 * len(data))
    val_size = int(0.15 * len(data))
    
    train_data = data[:train_size]
    val_data = data[train_size:train_size + val_size]
    test_data = data[train_size + val_size:]
    
    # 3. Crear datasets y dataloaders
    train_dataset = TimeSeriesDataset(train_data, config['window_size'], 
                                    config['prediction_steps'], config['epsilon'], train=True)
    val_dataset = TimeSeriesDataset(val_data, config['window_size'], 
                                  config['prediction_steps'], config['epsilon'], train=False)
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)
    
    # 4. Crear modelo
    model = ExtremeMemoryNetwork(
        input_size=1,
        hidden_size=config['hidden_size'],
        memory_size=config['memory_size'],
        window_size=config['window_size'],
        gamma=config['gamma']
    ).to(device)
    
    print(f'Número de parámetros: {sum(p.numel() for p in model.parameters()):,}')
    
    # 5. Entrenar modelo
    print("Comenzando entrenamiento...")
    train_losses, val_losses = train_model(
        model, train_loader, val_loader, 
        config['num_epochs'], config['learning_rate']
    )
    
    # 6. Evaluar y visualizar resultados
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    
    # ... (código para evaluación y visualización)

if __name__ == "__main__":
    main()